# Datenbereinigung und -vorbereitung
1. [Energie-Messwerte vorbereiten](#1)
1. [Innentempteratur vorbereiten](#2)
1. [Wetterdaten vorbereiten](#3)
1. [Feiertage vorbereiten](#4)
1. [Umfragedaten vorbereiten](#5)
1. [Daten zusammenführen](#6)

In [1]:
import numpy as np
import pandas as pd
import os
from datetime import date, timedelta

<a id="1"></a>
## 1. Energie-Messwerte vorbereiten
Gesamtverbrauch berechnen: [Pecan Street Inc. Data Use / Dataport FAQ](https://docs.google.com/document/d/1_9H9N4cgKmJho7hK8nii6flIGKPycL7tlWEtd4UhVEQ/edit#heading=h.nhqpscy1c9sm)

In [2]:
main_data = pd.read_csv('../input/2023-12-20/data_austin_15min.csv')

# Zeitstempel erstellen
main_data['timestamp'] = pd.to_datetime(main_data.local_15min.str.slice(0, 19))

# Gesamtverbrauch berechnen
main_data['use'] = main_data[['grid', 'solar']].sum(axis=1, skipna=True)

# Spalten entfernen
columns_to_drop = ['local_15min', 'grid', 'solar', 'leg1v', 'leg2v']
main_data.drop(columns=columns_to_drop, inplace=True)

# Stündlichen Durchschnitt bilden, damit später gejoint werden kann
main_data_hourly = main_data.groupby(['dataid', pd.Grouper(key='timestamp', freq='H')]).mean().reset_index()

main_data_hourly.sample(5)

,dataid,timestamp,air1,air2,air3,airwindowunit1,aquarium1,bathroom1,bathroom2,bedroom1,...,solar2,sprinkler1,sumppump1,utilityroom1,venthood1,waterheater1,waterheater2,wellpump1,winecooler1,use
77904,4031,2018-12-05 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.01325,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.23550
181503,8565,2018-10-17 06:00:00,0.00000,NaN,NaN,NaN,NaN,0.001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.77850
111759,6139,2018-10-26 13:00:00,0.00300,NaN,NaN,NaN,NaN,NaN,0.0015,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.06725
137968,7800,2018-10-26 01:00:00,0.00100,NaN,NaN,NaN,NaN,NaN,NaN,0.01150,...,NaN,NaN,NaN,NaN,0.018,NaN,NaN,NaN,NaN,0.34900
213741,9922,2018-06-24 03:00:00,0.71775,-0.0045,NaN,NaN,NaN,NaN,NaN,0.00500,...,NaN,NaN,NaN,0.0035,NaN,NaN,NaN,NaN,NaN,1.33650


<a id="2"></a>
## 2. Innentempteratur vorbereiten

In [3]:
# Daten aus mehreren Dateien zusammenführen
folder_path = '../input/2023-12-20/indoor_temp'
temp_dfs = list()

for file in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    temp_dfs.append(df)

indoor_temp_data = pd.concat(temp_dfs, ignore_index=True)

# Zeitstempel erstellen
indoor_temp_data['timestamp'] = pd.to_datetime(indoor_temp_data.localminute)

# Spalten auswählen
relevant_columns = ['dataid', 'timestamp', 'temp_c']
indoor_temp_data = indoor_temp_data[relevant_columns]

# Spalte umbenennen
indoor_temp_data.rename(columns={'temp_c': 'indoor_temp'}, inplace=True)

# Stündlichen Durchschnitt bilden, damit später gejoint werden kann
indoor_temp_hourly = indoor_temp_data.groupby(['dataid', pd.Grouper(key='timestamp', freq='H')]).mean().reset_index()

indoor_temp_hourly.sample(5)

,dataid,timestamp,indoor_temp
373022,8848,2014-08-14 10:00:00,22.573333
32834,484,2016-11-27 15:00:00,22.994000
341955,8029,2014-04-21 21:00:00,23.439500
162814,3310,2017-09-23 07:00:00,24.787000
238667,5252,2014-06-01 19:00:00,25.099000


<a id="3"></a>
## 3. Wetterdaten vorbereiten

In [4]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent='philipp')
location_cache = {}

def get_city(latitude, longitude):
    # Koordinaten als Schlüssel für den Cache verwenden
    coords_key = (latitude, longitude)
    
    # Überprüfen, ob Koordinaten bereits im Cache sind
    if coords_key in location_cache:
        city = location_cache[coords_key]
    else:
        # Koordinaten nicht im Cache, führe Reverse-Geocoding durch
        location = geolocator.reverse(f"{latitude}, {longitude}")
        city = location.raw['address'].get('city', 'unknown')  # 'unknown' als Fallback, falls keine Stadt gefunden wird
        location_cache[coords_key] = city
    
    return city

def convert_fahrenheit_to_celsius(fahrenheit):
    celsius = round((fahrenheit - 32) * 5 / 9, 2)
    return celsius

In [5]:
weather_data = pd.read_csv('../input/2023-12-20/weather.csv')

# Stadt anhand der Koordinaten herausfinden
weather_data['city'] = weather_data.apply(lambda row: get_city(row.latitude, row.longitude), axis=1)

# Zeilen filtern
weather_data = weather_data[weather_data.city == 'Austin']

# Zeitstempel erstellen
weather_data['timestamp'] = pd.to_datetime(weather_data.localhour)

# Umrechnung von Grad Fahrenheit zu Grad Celsius
weather_data['outdoor_temp'] = weather_data.temperature.apply(convert_fahrenheit_to_celsius)
weather_data['app_outdoor_temp'] = weather_data.apparent_temperature.apply(convert_fahrenheit_to_celsius)

# Spalten auswählen
relevant_columns = ['timestamp', 'outdoor_temp', 'app_outdoor_temp', 'humidity']
weather_data = weather_data[relevant_columns]

# Stündlichen Durchschnitt bilden, damit später gejoint werden kann
weather_data_hourly = weather_data.groupby(pd.Grouper(key='timestamp', freq='H')).mean().reset_index()

weather_data_hourly.sample(5)

,timestamp,outdoor_temp,app_outdoor_temp,humidity
38538,2015-05-25 18:00:00,22.42,22.42,0.89
69117,2018-11-19 21:00:00,10.47,10.47,0.67
27213,2014-02-07 21:00:00,1.54,-0.33,0.78
59750,2017-10-25 14:00:00,25.16,25.16,0.25
71899,2019-03-15 19:00:00,14.08,14.08,0.28


<a id="4"></a>
## 4. Feiertage vorbereiten
Feiertage in Austin: https://www.austintexas.gov/department/official-city-holidays

Die Feiertage umfassen:
* Neujahr (erster Wochentag im neuen Jahr)
* Martin-Luther-King-Tag (dritter Montag im Januar)
* Tag des Präsidenten (dritter Montag im Februar)
* Memorial Day (letzter Montag im Mai)
* Juneteenth (19. Juni)
* Unabhängigkeitstag (4. Juli)
* Tag der Arbeit (erster Montag im September)
* Veteranentag (11. November)
* Erntedankfest (vierter Donnerstag im November)
* Thanksgiving-Freitag (Tag nach Thanksgiving)
* Heiligabend (24. Dezember)
* Weihnachtstag (25. Dezember)

Anmerkung: Sollten Neujahr, der Unabhängigkeitstag oder Weihnachten auf einen Sonntag fallen, dann ist der Tag danach ebenfalls ein Feiertag. Wenn einer dieser Tage auf einen Samstag fällt, dann wird der Tag davor zum Feiertag (Quelle: https://usa.usembassy.de/feiertage.htm)

In [6]:
def find_specific_weekday_of_month(nth, weekday, month, year):
    """
    Funktion, um das spezifisches Vorkommen eines Wochentags in einem gegebenen Monat und Jahr zu finden.
    Wenn nth negativ ist, wird das letzte Vorkommen des Wochentags im Monat gefunden.

    :param nth: int, das Vorkommen des Wochentags im Monat. Negativ für das letzte Vorkommen.
    :param weekday: int, der Wochentag (0=Montag, 1=Dienstag, ..., 6=Sonntag).
    :param month: int, der Monat (1=Januar, 2=Februar, ..., 12=Dezember).
    :param year: int, das Jahr.
    :return: datetime.date, das Datum des spezifischen Vorkommens des Wochentags im angegebenen Monat und Jahr.
    """
    if nth < 0:
        # Ausgangsdatum ist der 1. des nächsten Monats
        if month == 12:
            start_date = date(year + 1, 1, 1)
        else:
            start_date = date(year, month + 1, 1)

        while start_date.weekday() != weekday:
            start_date -= timedelta(days=1)
        
        return start_date
    else:
        # Ausgangsdatum ist 1. des aktuellen Monats
        start_date = date(year, month, 1)

        while start_date.weekday() != weekday:
            start_date += timedelta(days=1)

        nth_weekday = start_date + timedelta(days=(nth - 1) * 7)
        return nth_weekday

In [7]:
def get_holidays(year):
    """
    Erstellt eine Liste der Feiertage für ein gegebenes Jahr in Austin.
    Die Funktion berücksichtigt spezielle Regeln für bewegliche Feiertage sowie Anpassungen für Feiertage,
    die auf das Wochenende fallen.

    :param year: int, das Jahr, für das die Feiertage berechnet werden sollen.
    :return: list, eine Liste mit Datumsangaben aller Feiertage für ein gegebenes Jahr in Austin.
    """
    holidays = {
        "Neujahrstag": date(year, 1, 1),
        "Martin-Luther-King-Tag": find_specific_weekday_of_month(3, 0, 1, year),
        "Tag des Präsidenten": find_specific_weekday_of_month(3, 0, 2, year),
        "Memorial Day": find_specific_weekday_of_month(-1, 0, 5, year),
        "Juneteenth": date(year, 6, 19),
        "Unabhängigkeitstag": date(year, 7, 4),
        "Tag der Arbeit": find_specific_weekday_of_month(1, 0, 9, year),
        "Veteranentag": date(year, 11, 11),
        "Erntedankfest": find_specific_weekday_of_month(4, 3, 11, year),
        "Thanksgiving-Freitag": find_specific_weekday_of_month(4, 3, 11, year) + timedelta(days=1),
        "Heiligabend": date(year, 12, 24),
        "Weihnachtstag": date(year, 12, 25)
    }

    # Anpassung für Feiertage, die auf das Wochenende fallen
    if holidays["Neujahrstag"].weekday() == 6:  # Sonntag
        holidays["Neujahrstag"] = date(year, 1, 2)
    if holidays["Unabhängigkeitstag"].weekday() in [5, 6]:  # Samstag oder Sonntag
        holidays["Unabhängigkeitstag"] = date(year, 7, 3) if holidays["Unabhängigkeitstag"].weekday() == 5 else date(year, 7, 5)
    if holidays["Weihnachtstag"].weekday() in [5, 6]:  # Samstag oder Sonntag
        holidays["Weihnachtstag"] = date(year, 12, 24) if holidays["Weihnachtstag"].weekday() == 5 else date(year, 12, 26)

    return holidays.values()

In [8]:
# Liste aller Tage von 2017 bis 2020 erstellen
start_date = date(2017, 1, 1)
end_date = date(2020, 12, 31)
delta = end_date - start_date
dates = [start_date + timedelta(days=i) for i in range(delta.days + 1)]

# Liste aller Feiertage von 2017 bis 2020 erstellen
holiday_dates = []
for year in range(2017, 2021):
    holiday_dates.extend(get_holidays(year))

data = {"date": [],
        "is_holiday": []}

for day in dates:
    data["date"].append(day)
    data["is_holiday"].append(day in holiday_dates)

# Datentyp ändern
holiday_data = pd.DataFrame(data)
holiday_data.date = pd.to_datetime(holiday_data.date)

holiday_data.head()

,date,is_holiday
0,2017-01-01,False
1,2017-01-02,True
2,2017-01-03,False
3,2017-01-04,False
4,2017-01-05,False


<a id="5"></a>
## 5. Umfragedaten vorbereiten
Ignorierte Daten u.a.:
* Demografische Daten über die Bewohner wie Ethie, Alter, Geschlecht, Bildungsabschluss, Einkommen, etc.
* Informationen aus Spalten, die nur als Freitext vorliegen und nicht direkt verwendbar sind wie Anzahl der Stunden im Home Office, Veränderungsarbeiten am Haus (wie z.B. Reparaturen, Austausch von Geräten, Neuinstallationen, Renovierungen)
* Details über Geräte wie Hersteller, typischer Stomverbrauch, etc.

In [9]:
def convert_square_feet_to_square_meters(feet):
    meters = round(feet * 0.092903, 2)
    return meters

In [10]:
survey_data = pd.read_csv('../input/2023-12-20/audits_surveys/survey_2013_all_participants.csv')

# Duplikate filtern
survey_data = survey_data.groupby('dataid').last().reset_index()

# Jahreszahlen einheitlich als Integer formatieren
survey_data['construction_year'] = survey_data.year_house_constructed.replace('1930 or earlier', '1930').astype('Int64')

# Datentypen ändern
survey_data['is_primary_residence'] = survey_data.primary_residence == "Yes"
survey_data['n_rooms'] = survey_data.house_num_rooms.astype('Int64')

# Anzahl der Bewohner über die Altersklassen berechnen
age_columns = ['residents_under_5', 'residents_6_to_12', 'residents_13_to_18', 'residents_19_to_24',
               'residents_25_to_34', 'residents_35_to_49', 'residents_50_to_64', 'residents_older_65']
survey_data['n_residents'] = survey_data[age_columns].sum(axis=1).astype('Int64')

# Fläche von Square Feet zu Quadratmeter umrechnen
survey_data['total_area'] = survey_data.house_square_feet.apply(convert_square_feet_to_square_meters)

# Spalten auswählen
relevant_columns = ['dataid', 'construction_year', 'is_primary_residence', 'n_residents', 'n_rooms', 'total_area']
survey_data = survey_data[relevant_columns]

survey_data.sample(5)

,dataid,construction_year,is_primary_residence,n_residents,n_rooms,total_area
240,8124,1955,True,5,7,292.64
278,9578,1972,True,1,5,76.64
241,8139,2005,True,2,6,153.29
158,5109,1979,True,3,8,371.61
210,7077,2012,True,4,11,325.16


<a id="6"></a>
## 6. Daten zusammenführen

In [11]:
# Fehlende Zeilen ohne Werte hinzufügen
main_data_hourly.set_index('timestamp', inplace=True)
main_data_full = main_data_hourly.groupby('dataid').apply(lambda group: group.asfreq('H')).drop(columns=['dataid']).reset_index()

# Umfragedaten joinen
# Teilnahme an der Umfrage aus 2013 ist Voraussetzung, da hier die meisten relevanten Informationen gesammelt wurden
j1 = pd.merge(main_data_full, survey_data, on='dataid', how='inner')

# Wetterdaten joinen
j2 = pd.merge(j1, weather_data_hourly, on='timestamp', how='left')

# Innentemperatur joinen
j3 = pd.merge(j2, indoor_temp_hourly, on='timestamp', how='left')

# Feiertage joinen
j3['date'] = pd.to_datetime(j3.timestamp.dt.date)
df = pd.merge(j3, holiday_data, on='date', how='left')
df.drop(columns=['date'], inplace=True)

# Spalte umbenennen
df.rename(columns={'dataid_x': 'dataid'}, inplace=True)

df.sample(5)

,dataid,timestamp,air1,air2,air3,airwindowunit1,aquarium1,bathroom1,bathroom2,bedroom1,...,is_primary_residence,n_residents,n_rooms,total_area,outdoor_temp,app_outdoor_temp,humidity,dataid_y,indoor_temp,is_holiday
56807,6139,2018-06-27 18:00:00,1.50450,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,True,5,9,260.13,35.97,37.97,0.37,NaN,NaN,False
82282,9922,2018-05-25 05:00:00,0.06925,-0.00475,NaN,NaN,NaN,NaN,NaN,0.11375,...,True,2,5,213.68,23.01,23.77,0.92,NaN,NaN,False
49930,5746,2018-09-14 05:00:00,0.24550,NaN,NaN,0.00475,NaN,NaN,NaN,NaN,...,True,4,9,164.25,22.70,23.49,0.95,NaN,NaN,False
66198,7800,2018-07-24 01:00:00,0.59800,NaN,NaN,NaN,NaN,NaN,NaN,0.00800,...,True,2,10,185.81,31.61,32.83,0.46,NaN,NaN,False
58805,6139,2018-09-19 00:00:00,0.00400,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,True,5,9,260.13,24.33,24.99,0.83,NaN,NaN,False


In [12]:
# Als csv-Datei speichern
df.to_csv('ps_ds1.csv', index=False)